In [2]:
#Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import random
import glob
import sys

In [4]:
def mean_pool(square_array,ratio):
  """
  Function to downsample a square array after applying a meanpool

  Inputs
  ------
    square_array : Array to be downsampled. Must be a square array with axes
                   lenghts that can be divisible by ratio
    ratio        : Downsampling ratio. i.e. a 1024x1024 array with a ratio of 4
                   will be downsampled to 256x256
  
  Outputs
  -------
    Returns the downsampled array
  """
  # Dimensions of array
  alen_1 = np.size(square_array,0)
  alen_2 = np.size(square_array,1)
  # Confirming array is square
  if (alen_1!=alen_2):
    print("ERROR: ARRAY NOT SQUARE")
  else:
    return square_array.reshape(int(alen_1/ratio), int(ratio), 
                                int(alen_1/ratio), int(ratio)).mean(axis=(1,3))


In [5]:
def convert_to_gz(fits_folder, gz_folder, down_sample_factor, date):
  """
  This script converts all fits files in a given folder to compressed 
  .gz numpy arrays, after preforming a meanpool downsample.

  Inputs
  ------
    fits_folder       : string folder path where the fits files to be 
                        downsampled are located
    gz_folder         : string folder path where the .gz files are to be 
                        stored after downsampling
    down_sample_factor: int representing the dimension reduction size for the 
                        downsample. (i.e. a down_sample_factor of 8 would turn 
                        4096x4096 fits files to 512x512 compressed arrays )
    date              : string showing the year and month during which the .fits
                        files were taken.
  """
  # Check how many files need to be run
  # Glob returns list of files with a certain file extension
  fits_list = glob.glob(fits_folder+'*.fits')
  num_fits = len(fits_list)

  # Glob returns list of files with a certain file extension
  gz_list = glob.glob(gz_folder+'*'+date+'*.gz')
  num_gz = len(gz_list)

  total_files = num_fits - num_gz #(total number of files remaining)
  print("%d fits files in "%num_fits+fits_folder)
  print("%d gz files in "%num_gz+gz_folder)
  if total_files < 1:
    print("All files converted, unless duplicate gz files exist.\n")
    return
  else:
    print("%d files remaining"%total_files)


  # Iterating over each file
  counter = 0
  for filename in fits_list:
    filename = os.path.basename(filename)
    if not os.path.exists(gz_folder+filename[:-5]+".gz"):
      #opening file
      fits_file = fits.open( fits_folder+filename , dtype=np.int16 )
      #saving downsampled file
      np.savetxt(X=mean_pool(np.array(fits_file[1].data),down_sample_factor),
                fname=gz_folder+filename[:-5]+".gz")
      if counter%10 == 0:
        print("%.3f %% finished"%(100.0*counter/total_files))
      counter += 1

In [7]:
##### Script
AIA_path = "/content/drive/Shareddrives/Phys 477 - Astro Project/AIA_files/"
event_folder = 'XRA_events/'
null_folder = 'XRA_nulls/'

wavelengths = (131, 171, 211)
date_paths = ["2020_01_"]

for w in wavelengths:
  for date in date_paths:
    event_path = AIA_path + event_folder + date + str(w) +"/"
    null_path = AIA_path + null_folder + date + str(w) +"/"

    event_gz_path = AIA_path + event_folder + "gz_" + str(w) +"_1024/"
    null_gz_path = AIA_path + null_folder + "gz_" + str(w) +"_1024/"

    print("Converting path: " + event_folder + date + str(w) +"/ to gz_1024")
    convert_to_gz(event_path, event_gz_path, 4, date[:-1])
    print("Converting path: " + null_folder + date + str(w) +"/ to gz_1024")
    convert_to_gz(null_path, null_gz_path, 4, date[:-1])
print("\nAll Conversions Complete.")


Converting path: XRA_events/2020_01_131/ to gz_1024
1 fits files in /content/drive/Shareddrives/Phys 477 - Astro Project/AIA_files/XRA_events/2020_01_131/
0 gz files in /content/drive/Shareddrives/Phys 477 - Astro Project/AIA_files/XRA_events/gz_131_1024/
1 files remaining
0.000 % finished
Converting path: XRA_nulls/2020_01_131/ to gz_1024
1 fits files in /content/drive/Shareddrives/Phys 477 - Astro Project/AIA_files/XRA_nulls/2020_01_131/
0 gz files in /content/drive/Shareddrives/Phys 477 - Astro Project/AIA_files/XRA_nulls/gz_131_1024/
1 files remaining
0.000 % finished
Converting path: XRA_events/2020_01_171/ to gz_1024
1 fits files in /content/drive/Shareddrives/Phys 477 - Astro Project/AIA_files/XRA_events/2020_01_171/
0 gz files in /content/drive/Shareddrives/Phys 477 - Astro Project/AIA_files/XRA_events/gz_171_1024/
1 files remaining
0.000 % finished
Converting path: XRA_nulls/2020_01_171/ to gz_1024
1 fits files in /content/drive/Shareddrives/Phys 477 - Astro Project/AIA_files/